# (Homework) Week 6 - DataScience Bootcamp Fall 2025

All solution cells are replaced with `# TODO` placeholders so you can fill them in.

**Name:** Claudia Duran Garcia \
**Email:** cmd9763@nyu.edu

---

### Problem 1: Dataset Splitting

1. You have recordings of 44 phones from 100 people; each person records ~200 phones/day for 5 days.
   - Design a valid training/validation/test split strategy that ensures the model generalizes to **new speakers**.

2. You now receive an additional dataset of 10,000 phone recordings from **Kilian**, a single speaker.
   - You must train a model that performs well **specifically for Kilian**, while also maintaining generalization.

*Describe your proposed split strategy and reasoning.* (Theory)

In [ ]:
# The train/test/split strategy consists of splitting the dataset into three parts:
# a training set, a validation set, and a test set. The training set is used to train the model,
# the validation set is used to tune hyperparameters and make decisions about the model, 
# and the test set is used to evaluate the final performance of the model.

# Number of recordings in the dataset: 200 recordings * 44 phones * 100 people * 5 days = 4,400,000 recordings

# In this example, we will use a 70/15/15 split for training, validation, and testing respectively.
# For Kilian, we will use a 60/20/20 split to ensure enough data for validation and testing because
# it has more recordings for a single person.


In [21]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("Libraries imported successfully!")

Libraries imported successfully!


### Problem 2: K-Nearest Neighbors

1. **1-NN Classification:** Given dataset:

   Positive: (1,2), (1,4), (5,4)

   Negative: (3,1), (3,2)

   Plot the 1-NN decision boundary and classify new points visually.

2. **Feature Scaling:** Consider dataset:

   Positive: (100,2), (100,4), (500,4)

   Negative: (300,1), (300,2)

   What would the 1-NN classify point (500,1) as **before and after scaling** to [0,1] per feature?

3. **Handling Missing Values:** How can you modify K-NN to handle missing features in a test point?


4. **High-dimensional Data:** Why can K-NN still work well for images even with thousands of pixels?


In [15]:
# Demonstrate effect of k on decision boundary
from matplotlib.colors import ListedColormap

def plot_decision_boundary(k_value=3):
    # Create mesh
    h = 0.2
    x_min, x_max = -0.5, 10.5
    y_min, y_max = -0.5, 10.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Train k-NN
    knn = KNeighborsClassifier(n_neighbors=k_value)
    knn.fit(X, y)
    
    # Predict on mesh
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot
    plt.figure(figsize=(10, 8))
    cmap_light = ListedColormap(['#FFAAAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#0000FF'])
    
    plt.contourf(xx, yy, Z, cmap=cmap_light, alpha=0.4)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
               s=100, edgecolor='black', alpha=0.7)
    
    plt.xlabel('X bound', fontsize=13)
    plt.ylabel('Y bound', fontsize=13)
    plt.title(f'k-NN Decision Boundary (k={k_value})', fontsize=15, fontweight='bold')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    
    # Add legend
    plt.scatter([], [], c='red', s=100, label='Negative Region', alpha=0.4)
    plt.scatter([], [], c='blue', s=100, label='Positive Region', alpha=0.4)
    plt.legend(fontsize=11)
    
    plt.tight_layout()
    plt.show()
    
    if k_value == 1:
        print("  k=1: Very jagged boundary - overfitting to noise!")
    elif k_value > 10:
        print("  Large k: Very smooth boundary - might be too simple!")
    else:
        print(" Good k value: Balanced decision boundary!")

In [16]:
# 1-NN Classification
X = np.array([[1, 2],
              [1, 4],
              [5, 4],
              [3, 1],
              [3, 2]])
# Labels: 1 = Positive, 0 = Negative
y = np.array([1, 1, 1, 0, 0])

interact(plot_decision_boundary,
         k_value=IntSlider(min=1, max=15, step=1, value=3, description='k:'))

interactive(children=(IntSlider(value=3, description='k:', max=15, min=1), Output()), _dom_classes=('widget-in…

<function __main__.plot_decision_boundary(k_value=3)>

In [ ]:
# Feature Scaling
X_big = np.array([[100, 2],
                  [100, 4],
                  [500, 4],
                  [300, 1],
                  [300, 2]])
y_big = np.array([1, 1, 1, 0, 0])  # 1 = Positive, 0 = Negative

query = np.array([[500, 1]])  # point to classify

# 1-NN before scaling
knn_raw = KNeighborsClassifier(n_neighbors=1)
knn_raw.fit(X_big, y_big)
pred_raw = knn_raw.predict(query)
print("Before scaling: prediction =", int(pred_raw[0]))  # 1 or 0

# Scale each feature to [0, 1] using MinMax scaling (fit on training data)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X_big)
query_scaled = scaler.transform(query)

print("Feature mins:", scaler.data_min_)
print("Feature maxs:", scaler.data_max_)
print("Query scaled:", query_scaled)

# 1-NN after scaling
knn_scaled = KNeighborsClassifier(n_neighbors=1)
knn_scaled.fit(X_scaled, y_big)
pred_scaled = knn_scaled.predict(query_scaled)
print("After MinMax scaling: prediction =", int(pred_scaled[0]))

# Manual min-max formula (for reference)
# x_scaled = (x - min) / (max - min)
mins = scaler.data_min_
maxs = scaler.data_max_
manual_scaled = (query - mins) / (maxs - mins)
print("Manual scaled (should match):", manual_scaled)

Before scaling: prediction = 1
Feature mins: [100.   1.]
Feature maxs: [500.   4.]
Query scaled: [[1. 0.]]
After MinMax scaling: prediction = 0
Manual scaled (should match): [[1. 0.]]


In [19]:
# 3. Handling Missing Values: How can you modify K-NN to handle missing features in a test point?
# You can modify K-NN to handle missing features by using distance metrics that can ignore missing values,
# such as using only the available features to compute distances.

# 4. High-dimensional Data: Why can K-NN still work well for images even with thousands of pixels?
# You can use dimensionality reduction techniques (like PCA) to reduce the number of features while retaining
# most of the variance in the data, making K-NN more effective in high-dimensional spaces.

### Problem 3: Part 1

You are given a fully trained Perceptron model with weight vector **w**, along with training set **D_TR** and test set **D_TE**.

1. Your co-worker suggests evaluating $h(x) = sign(w \cdot x)$ for every $(x, y)$ in D_TR and D_TE. Does this help determine whether test error is higher than training error?
2. Why is there no need to compute training error explicitly for the Perceptron algorithm?

In [20]:
# You are given a fully trained Perceptron model with weight vector **w**, along with training set **D_TR** and test set **D_TE**.

# 1. Your co-worker suggests evaluating $h(x) = sign(w \cdot x)$ for every $(x, y)$ in D_TR and D_TE. Does this help determine whether test error is higher than training error?
# No, evaluating $h(x) = sign(w . x)$ for every $(x, y)$ in D_TR and D_TE will give you the predictions for both sets, but it does not directly provide information about the error rates. 
# To determine whether the test error is higher than the training error, you would need to compare the predicted labels to the actual labels in both datasets and calculate the error rates 
# (i.e., the proportion of misclassified examples) for each set.

# 2. Why is there no need to compute training error explicitly for the Perceptron algorithm?
# The Perceptron algorithm updates its weights based on misclassified training examples during the training process.
# Therefore, if the training process has converged, it implies that the training error is zero

### Problem 3: Two-point 2D Dataset (Part 2)

Run the Perceptron algorithm **by hand or in code** on the following data:

1. Positive class: (10, -2)
2. Negative class: (12, 2)

Start with $w_0 = (0, 0)$ and a learning rate of 1.

- Compute how many updates are required until convergence.
- Write down the sequence of $w_i$ vectors.

In [23]:
# Perceptron implementation
# y = mx + c -> y = wx + b
class Perceptron:
    def __init__(self):
        self.w = None
        self.b = 0
        self.history = []
    
    def predict(self, x):
        return np.sign(np.dot(self.w, x) + self.b)
    
    def train_step(self, x, y):
        if self.w is None:
            self.w = np.zeros(len(x))
        
        prediction = self.predict(x)
        
        # Save state before update
        self.history.append({
            'w': self.w.copy(),
            'b': self.b,
            'x': x,
            'y': y,
            'pred': prediction,
            'correct': prediction == y
        })
        
        # Update if wrong
        if prediction != y:
            self.w += y * x
            self.b += y
            return False  # Made an update
        return True  # Correct prediction

print("Perceptron class defined!")

Perceptron class defined!


In [22]:
# Todo
# Generate linearly separable data
np.random.seed(42)

# Positive class
X_pos = np.random.randn(10, 2) + np.array([-2, -2])
y_pos = np.ones(20)

# Negative class
X_neg = np.random.randn(12, 2) + np.array([2, 2])
y_neg = -np.ones(20)

# Combine
X_perceptron = np.vstack([X_pos, X_neg])
y_perceptron = np.concatenate([y_pos, y_neg])

print(f"✅ Generated linearly separable dataset:")
print(f"   - {len(X_pos)} positive examples")
print(f"   - {len(X_neg)} negative examples")

✅ Generated linearly separable dataset:
   - 10 positive examples
   - 12 negative examples


In [27]:
# Train perceptron and visualize
perceptron = Perceptron()

print("🎓 Training Perceptron...\n")
print("="*60)

max_epochs = 10
for epoch in range(max_epochs):
    errors = 0
    for i, (x, y) in enumerate(zip(X_perceptron, y_perceptron)):
        correct = perceptron.train_step(x, y)
        if not correct:
            errors += 1
    
    print(f"Epoch {epoch + 1}: {errors} errors, w={perceptron.w}, b={perceptron.b:.2f}")
    
    if errors == 0:
        print(f"\n🎉 Converged after {epoch + 1} epochs!")
        break

print("="*60)

🎓 Training Perceptron...

Epoch 1: 3 errors, w=[-0.77610366 -2.53540888], b=1.00
Epoch 2: 2 errors, w=[-0.04892147 -2.93255346], b=1.00
Epoch 3: 2 errors, w=[ 0.67826072 -3.32969805], b=1.00
Epoch 4: 2 errors, w=[ 1.40544291 -3.72684263], b=1.00
Epoch 5: 2 errors, w=[ 2.1326251  -4.12398721], b=1.00
Epoch 6: 4 errors, w=[ 0.36371746 -2.03105102], b=1.00
Epoch 7: 2 errors, w=[ 1.09089965 -2.4281956 ], b=1.00
Epoch 8: 2 errors, w=[-0.19194965 -2.48864129], b=1.00
Epoch 9: 2 errors, w=[ 0.53523254 -2.88578588], b=1.00
Epoch 10: 2 errors, w=[ 1.26241473 -3.28293046], b=1.00


### Problem 4: Reconstructing the Weight Vector

Given the log of Perceptron updates:

| x | y | count |
|---|---|--------|
| (0, 0, 0, 0, 4) | +1 | 2 |
| (0, 0, 6, 5, 0) | +1 | 1 |
| (3, 0, 0, 0, 0) | -1 | 1 |
| (0, 9, 3, 6, 0) | -1 | 1 |
| (0, 1, 0, 2, 5) | -1 | 1 |

Assume learning rate = 1 and initial weight $w_0 = (0, 0, 0, 0, 0)$.

Compute the final weight vector after all updates.

In [31]:
#Todo


### Problem 5: Visualizing Perceptron Convergence

Implement a Perceptron on a small 2D dataset with positive and negative examples.

- Plot the data points.
- After each update, visualize the decision boundary.
- Show how it converges to a stable separator.

In [28]:
# Interactive perceptron visualization
def visualize_perceptron_step(step=0):
    if step >= len(perceptron.history):
        step = len(perceptron.history) - 1
    
    state = perceptron.history[step]
    w = state['w']
    b = state['b']
    
    plt.figure(figsize=(12, 8))
    
    # Plot data points
    plt.scatter(X_pos[:, 0], X_pos[:, 1], c='blue', s=100, alpha=0.6, 
               label='Positive (+1)', edgecolors='black')
    plt.scatter(X_neg[:, 0], X_neg[:, 1], c='red', s=100, alpha=0.6, 
               label='Negative (-1)', edgecolors='black')
    
    # Highlight current point
    current_x = state['x']
    current_y = state['y']
    color = 'green' if state['correct'] else 'orange'
    plt.scatter(current_x[0], current_x[1], c=color, s=400, marker='*',
               edgecolors='black', linewidths=2, zorder=5,
               label='Current Point')
    
    # Plot decision boundary if weights are non-zero
    if np.any(w != 0):
        x_line = np.linspace(-5, 5, 100)
        # w1*x1 + w2*x2 + b = 0  =>  x2 = -(w1*x1 + b)/w2
        if w[1] != 0:
            y_line = -(w[0] * x_line + b) / w[1]
            plt.plot(x_line, y_line, 'g-', linewidth=3, label='Decision Boundary')
            
            # Plot normal vector (direction of w)
            origin = np.array([0, -b/w[1]]) if w[1] != 0 else np.array([-b/w[0], 0])
            plt.arrow(origin[0], origin[1], w[0]*0.5, w[1]*0.5, 
                     head_width=0.3, head_length=0.2, fc='purple', ec='purple',
                     linewidth=2, label='Weight Vector')
    
    plt.xlabel('Feature 1', fontsize=13)
    plt.ylabel('Feature 2', fontsize=13)
    plt.title(f'Perceptron Learning - Step {step + 1}/{len(perceptron.history)}\n' +
             f'w=[{w[0]:.2f}, {w[1]:.2f}], b={b:.2f}\n' +
             f'Prediction: {int(state["pred"])}, True: {int(current_y)} - ' +
             ('✓ Correct' if state['correct'] else '✗ Wrong, Updating!'),
             fontsize=14, fontweight='bold')
    plt.legend(fontsize=10, loc='upper right')
    plt.grid(True, alpha=0.3)
    plt.xlim(-5, 5)
    plt.ylim(-5, 5)
    plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5, alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', linewidth=0.5, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📍 Step {step + 1} Details:")
    print(f"   Current point: x = {current_x}")
    print(f"   True label: y = {int(current_y)}")
    print(f"   Prediction: ŷ = {int(state['pred'])}")
    print(f"   Score: w·x + b = {np.dot(w, current_x) + b:.2f}")
    
    if not state['correct']:
        new_w = w + current_y * current_x
        new_b = b + current_y
        print(f"   ❌ Wrong prediction! Updating...")
        print(f"   New w: {w} + {current_y} × {current_x} = {new_w}")
        print(f"   New b: {b:.2f} + {current_y} = {new_b:.2f}")
    else:
        print(f"   ✅ Correct! No update needed.")

# Create interactive widget
interact(visualize_perceptron_step,
         step=IntSlider(min=0, max=len(perceptron.history)-1, step=1, value=0,
                       description='Step:', continuous_update=False))

interactive(children=(IntSlider(value=0, continuous_update=False, description='Step:', max=219), Output()), _d…

<function __main__.visualize_perceptron_step(step=0)>